# Выделение ключевых фраз. Отзывы (1-110 пользователь)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_rows = 10

## Выгрузка, группировка и склейка отзывов

Отзывы предварительно приведены к нижнему регистру, очищены от пунктуации, лишних пробелов и стоп-слов, лемматизированы

In [6]:
electives = pd.read_csv('drive/MyDrive/ОБЩЕЕ/Отзывус/AI/key_tokens/data/feedbacks_dropna_sorted_cleaned.csv', sep=';')
electives

,На какой электив вы хотите оставить отзыв?,Какие ещё названия этого электива вы знаете?,На сколько бы вы оценили этот электив?,Введите текст отзыва на электив,Чем вы руководствовались при выборе этого электива?,Чего вы ожидали от этого электива?,Ваши ожидания от этого электива оправдались?,Что дало вам прохождение этого электива?,Какой преподаватель вёл этот электив?,На сколько бы вы оценили этого преподавателя в рамках этого электива?
0,3D и медийная реклама в программе 3DS MAX,"3д, 3d, 3 д, 3 d, 3д моделирование, 3д проекти...",4,"Чисто основы основ 3д моделинга, для тех кто в...",Описанием электива в карточке МУП;Названием эл...,"Введение в основы 3д моделинга, постановки сце...","Скорее да, чем нет",Определенно дал какие-то навыки в 3д. Теперь е...,Мокроусов Сергей Иванович,5.0
1,Python - программная среда для аналитиков,NaN,5,Кайфы ловлю,Названием электива;Тематикой электива;,Кайфов,Да,Кайфы,Актаев Нуркен Ерболатович,5.0
2,R для лингвистов: программирование и анализ да...,Программирование в R,1,Проходил он онлайн на платформе openedu.ru. По...,Попал случайно;,Я совсем не хотела его проходить,Нет,"Ничего, только нервы помотал",Курс шёл онлайн,2.0
3,Адаптация населения России к трудностям револю...,Адаптация,5,Супер пушка класс препод супер интересные исто...,Тематикой электива;Списком преподавателей;Отзы...,То что и получил,Да,Большой пенис,Кононенко Анатолий Анатольевич,5.0
4,Анализ данных в R,NaN,4,"Анализируем данные на ЯП R, на автомат выйти с...",Наугад;,NaN,"Скорее нет, чем да",NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...
192,Экология. Технологии. Аниме/Ecology. Technolog...,NaN,4,"Плюс в том, что можешь пройти когда хочешь, ми...",Описанием электива в карточке МУП;Названием эл...,Просмотра интересного аниме,"Скорее нет, чем да",NaN,NaN,4.0
193,Эристика,NaN,5,"Помимо коммуникативных навыков, даёт неплохой ...",Тематикой электива;Отзывами других студенов;,"что мои навыки ""спорщика"" улучшатся",Да,Многие уже имеющиеся знания были систематизиро...,NaN,5.0
194,Эристика,NaN,4,"Электив, на котором учат искусству спора и пуб...",Описанием электива в карточке МУП;Названием эл...,NaN,Да,Непобедимость в спорах любой сложности. А вооб...,Пупышева Ирина Николаевна,5.0
195,Эристика,NaN,5,Классный электив для любителей поработать язык...,Списком преподавателей;Отзывами других студено...,Интересные дискуссии,Да,"Понимание того, что главное в споре – не выясн...",Пупышева Ирина Николаевна,5.0


In [35]:
print("Элективов:", len(electives['На какой электив вы хотите оставить отзыв?'].unique()))

symbols = electives['Введите текст отзыва на электив'].apply(lambda t: len(t))
print("Средняя длинна отзыва:", symbols.mean())
print("min/max:", symbols.min(), "|", symbols.max())

words = electives['Введите текст отзыва на электив'].str.split().apply(len)
print("Среднее кол-во слов в отзыве:", words.mean())
print("min/max:", words.min(), "|", words.max())

Элективов: 132
Средняя длинна отзыва: 319.93908629441626
min/max: 11 | 1354
Среднее кол-во слов в отзыве: 47.213197969543145
min/max: 2 | 222


In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

In [ ]:
def series_concat(series: pd.core.series.Series):
  text = ""
  for i in series:
    text += i + " "
  text = text.lower()
  return text

In [ ]:
!pip install -q gwpy

In [ ]:
%%capture
!pip install natasha

from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

In [ ]:
nat_segmenter = Segmenter()
nat_morph_vocab = MorphVocab()

nat_emb = NewsEmbedding()
nat_morph_tagger = NewsMorphTagger(nat_emb)
nat_syntax_parser = NewsSyntaxParser(nat_emb)
nat_ner_tagger = NewsNERTagger(nat_emb)

def lem_by_natasha_stopworded(text: str):
  text = text.lower()
  tokens = "".join([lit if lit not in punctuation else ' ' for lit in text]).split(" ")
  tokens = [token for token in tokens if len(token) > 0 and token not in russian_stopwords]
  text = " ".join(tokens)

  doc = Doc(text)
  doc.segment(nat_segmenter)
  doc.tag_morph(nat_morph_tagger)
  doc.parse_syntax(nat_syntax_parser)
  doc.tag_ner(nat_ner_tagger)

  new_text = ""
  for token in doc.tokens:
      token.lemmatize(nat_morph_vocab)
      new_text += token.lemma + " "

  return new_text

In [ ]:
%%time
electives_concat_feedbacks = pd.DataFrame(electives.groupby(["На какой электив вы хотите оставить отзыв?"])["Введите текст отзыва на электив"].apply(series_concat))
electives_concat_feedbacks["stopworded_lem_text"] = electives_concat_feedbacks["Введите текст отзыва на электив"].apply(lem_by_natasha_stopworded)
electives_concat_feedbacks["count_of_feedbacks"] = electives["На какой электив вы хотите оставить отзыв?"].value_counts().sort_index()
electives_concat_feedbacks.reset_index(drop=False, inplace=True)
electives_concat_feedbacks.columns = ["title", "concat_text", "stopworded_lem_text", "count"]
electives_concat_feedbacks

CPU times: user 2min 45s, sys: 46.1 s, total: 3min 31s
Wall time: 3min 15s


,title,concat_text,stopworded_lem_text,count
0,3D и медийная реклама в программе 3DS MAX,"чисто основы основ 3д моделинга, для тех кто в...",чисто основа основа 3 д моделинг тот шарит ску...,1
1,Python - программная среда для аналитиков,кайфы ловлю,кайф ловля,1
2,R для лингвистов: программирование и анализ да...,проходил он онлайн на платформе openedu.ru. по...,проходить онлайн платформа openedu ru попасть ...,1
3,Адаптация населения России к трудностям револю...,супер пушка класс препод супер интересные исто...,супер пушка класс препод супер интересный исто...,1
4,Анализ данных в R,"анализируем данные на яп r, на автомат выйти с...",анализировать данные яп r автомат выйти сложно...,3
...,...,...,...,...
127,Экологические проблемы Тюменского региона: миф...,"проходит в соцгуме. преподы супер, у нас было ...",проходить соцгум препод супер несколько препод...,1
128,Экология,"бесполезен, если вы не заинтересованы в ней. п...",бесполезный заинтересованы проходить онлайн по...,2
129,Экология. Технологии. Аниме/Ecology. Technolog...,"занятий не было, просто давали просмотреть ани...",занятие просто давать просмотреть аниме платфо...,3
130,Эристика,"помимо коммуникативных навыков, даёт неплохой ...",помимо коммуникативный навык давать неплохой т...,3


In [ ]:
electives_concat_feedbacks.to_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/key_tokens/data/feedbacks_dropna_sorted_cleaned_stopsloved_natashed.csv")

## ИЛИ Загрузка датасета, прошедшего через все круги Наташи и проч.

In [ ]:
electives_concat_feedbacks = pd.read_csv('drive/MyDrive/ОБЩЕЕ/Отзывус/AI/key_tokens/data/feedbacks_dropna_sorted_cleaned_stopsloved_natashed.csv')
electives_concat_feedbacks

,Unnamed: 0,title,concat_text,stopworded_lem_text,count
0,0,3D и медийная реклама в программе 3DS MAX,"чисто основы основ 3д моделинга, для тех кто в...",чисто основа основа 3 д моделинг тот шарит ску...,1
1,1,Python - программная среда для аналитиков,кайфы ловлю,кайф ловля,1
2,2,R для лингвистов: программирование и анализ да...,проходил он онлайн на платформе openedu.ru. по...,проходить онлайн платформа openedu ru попасть ...,1
3,3,Адаптация населения России к трудностям револю...,супер пушка класс препод супер интересные исто...,супер пушка класс препод супер интересный исто...,1
4,4,Анализ данных в R,"анализируем данные на яп r, на автомат выйти с...",анализировать данные яп r автомат выйти сложно...,3
...,...,...,...,...,...
127,127,Экологические проблемы Тюменского региона: миф...,"проходит в соцгуме. преподы супер, у нас было ...",проходить соцгум препод супер несколько препод...,1
128,128,Экология,"бесполезен, если вы не заинтересованы в ней. п...",бесполезный заинтересованы проходить онлайн по...,2
129,129,Экология. Технологии. Аниме/Ecology. Technolog...,"занятий не было, просто давали просмотреть ани...",занятие просто давать просмотреть аниме платфо...,3
130,130,Эристика,"помимо коммуникативных навыков, даёт неплохой ...",помимо коммуникативный навык давать неплохой т...,3


## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

Векторизация всего корпуса склееных отзывов

In [ ]:
cv = CountVectorizer()
word_count_vector = cv.fit_transform(electives_concat_feedbacks["stopworded_lem_text"])
word_count_vector.shape

(132, 2046)

Вычисление TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(electives_concat_feedbacks["stopworded_lem_text"])

TF-IDF для конкретно взятого электива

In [ ]:
def tfidf_pred(el_index: int):
  document = tfidf_vectorizer.transform([lem_by_natasha_stopworded(electives_concat_feedbacks["concat_text"][el_index])])
  df = pd.DataFrame(document.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
  df = df.sort_values(by=["tfidf"], ascending=False)
  return ", ".join([i for i in df.head(10).index])

In [ ]:
%%time
tfidf_pred(5)

CPU times: user 49.7 ms, sys: 22.4 ms, total: 72.1 ms
Wall time: 81.1 ms


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'данные, ами, приобрести, чат, банк, сравнение, телеграмма, пользоваться, огромный, рамках'

## YAKE
https://doi.org/10.1016/j.ins.2019.09.013

In [ ]:
!pip install yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 KB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp38-cp38-linux_x86_64.whl size=70618 sha256=3d7d6063e07a798ffca4db8e448ed4abe04ad96db8eb60dc0b9eaba8d71751ef
  Stored in directory: /root/.cache/pip/wheels/f1/c7/3c/4c83132de76359e3a429fd09c08995945ca96c5290a41651d3
Successfully built jellyfish


In [ ]:
import yake

In [ ]:
yake_extractor = yake.KeywordExtractor(
    lan = "ru",
    n = 5,
    dedupLim = 0.3,
    top = 10
)

In [ ]:
def yake_pred(el_index: int):
  result = ""
  for i in yake_extractor.extract_keywords(electives_concat_feedbacks["concat_text"][el_index])[::-1]:
     result += i[0] + ", "
  return result

In [ ]:
%%time
yake_pred(5)

CPU times: user 304 ms, sys: 3.62 ms, total: 308 ms
Wall time: 396 ms


'excel, сайты с определенными данными, какими банками данных пользоваться, достаточно информации в рамках электива, огромным плюсом по сравнению, сравнению с другими моок-ами, задавать вопросы по заданиям, имелся чат в телеграмме, приобрела новые навыки в excel, легко закрыться, '

## RAKE
https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents
https://github.com/csurfer/rake-nltk

ОЧИСТКА ОТ ПУНКТУАЦИИ?!

In [ ]:
!pip install rake-nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
from string import punctuation
punct = [i for i in punctuation]
punct

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from rake_nltk import Rake
r = Rake(min_length=2, punctuations=punctuation, language="russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def rake_pred(el_index: int):
  result = ""
  r.extract_keywords_from_text(electives_concat_feedbacks["concat_text"][el_index])
  for i in r.get_ranked_phrases()[:10]:
     result += i + ", "
  return result

In [ ]:
%%time
rake_pred(5).split(",")

CPU times: user 13 ms, sys: 2.86 ms, total: 15.9 ms
Wall time: 32.4 ms


['какими банками данных пользоваться',
 ' приобрела новые навыки',
 ' предоставлено достаточно информации',
 ' рамках электива',
 ' определенными данными',
 ' огромным плюсом',
 ' некоторыми заданиями',
 ' нам нужно',
 ' найти сайты',
 ' легко закрыться',
 ' ']

## python keyphrase extraction (PKE): TextRank

In [ ]:
%%capture
!pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download ru_core_news_md

In [ ]:
import pke, ru_core_news_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
!pip install scipy==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [ ]:
def textrank_pred(el_index: int):
  result = ""
  extractor = pke.unsupervised.TextRank()
  extractor.load_document(input=electives_concat_feedbacks["concat_text"][el_index], language="ru")
  extractor.candidate_selection()
  extractor.candidate_weighting()
  for i in extractor.get_n_best(n=10):
     result += i[0] + ", "
  return result

In [ ]:
%%time
textrank_pred(5)

CPU times: user 2.63 s, sys: 783 ms, total: 3.41 s
Wall time: 3.43 s


'банками данных, определенными данными, рамках электива, другими моок-ами, огромным плюсом, новые навыки, информации, сайты, нужно, проблемы, '

## KeyBERT
https://doi.org/10.5281/zenodo.4461265

In [ ]:
%%capture
!pip install transformers sentencepiece

In [ ]:
import torch
from transformers import AutoModel
# bert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
bert_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%capture
!pip install keybert

In [ ]:
from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT(model=bert_model)

In [ ]:
def keybert_pred(el_index: int):
  result = ""
  for i in kw_model.extract_keywords(electives_concat_feedbacks["concat_text"][el_index],
                                     keyphrase_ngram_range=(3, 6),
                                     stop_words=None,
                                     use_mmr=True, diversity=0.9,
                                     top_n=10)[::-1]:
    result += i[0] + ", "
  return result

In [ ]:
%%time
keybert_pred(130).split(",")

CPU times: user 231 ms, sys: 12.5 ms, total: 243 ms
Wall time: 239 ms


['довольно интересная штука классный',
 ' разбирали различные виды',
 ' на котором учат искусству',
 ' дискуссий стратегии тактики',
 ' для любителей поработать языком',
 ' выступлениям придется много',
 ' даёт неплохой толчок для развития логики',
 ' навыков даёт неплохой толчок для развития',
 ' котором учат искусству спора',
 ' так же психологии общения электив',
 ' ']

## Сравнение методов по различным элективам

In [ ]:
electives_concat_feedbacks.sort_values("count", ascending=False)["count"]

57     6
82     5
92     5
95     5
124    5
      ..
47     1
46     1
44     1
42     1
131    1
Name: count, Length: 132, dtype: int64

In [ ]:
sum(electives_concat_feedbacks.sort_values("count", ascending=False)["count"])

197

In [ ]:
el_indeces = list(electives_concat_feedbacks.sort_values("count", ascending=False).index)
predictors = {"TFIDF": tfidf_pred,
              "YAKE": yake_pred,
              "TextRank": textrank_pred,
              "KeyBERT": keybert_pred,
              "RAKE": rake_pred}

In [ ]:
el_indeces = list(electives_concat_feedbacks.sort_values("count", ascending=False).index)
predictors = {"KeyBERT": keybert_pred}

In [ ]:
predictions = {"Titles": [],
               "Feedbacks": [],
               "TFIDF": [],
              "YAKE": [],
              "TextRank": [],
              "KeyBERT": [],
               "RAKE": []}
for el_index in el_indeces:
  predictions["Titles"].append(electives_concat_feedbacks["title"][el_index])
  predictions["Feedbacks"].append(electives_concat_feedbacks["concat_text"][el_index])
  for predictor_name, predictor_method in predictors.items():
    predictions[predictor_name].append(predictor_method(el_index))

predictions = pd.DataFrame(predictions)
predictions

,Titles,Feedbacks,TFIDF,YAKE,TextRank,KeyBERT,RAKE
0,Литература фанфикшн: интерпретация классически...,"обсуждение канона, фанона, фанфиков, пейрингов...","фанфик, лояльный, петров, литература, простой,...","пары, интересно, фанфиков, используемых для ан...",преподавателя - петрова владимира владимирович...,"вкидывать свое мнение предлагать вселенную, ли...","чтение фанфиков стало учёбой ?), которой пишут..."
1,Практическая риторика и теория аргументации,"преподаватель очень добрая девушка, любит свой...","выступление, выступать, очень, риторика, пожал...","страх сцены, электив, раскрепощенными в общени...","следующие практические пары, подробной обратно...","тему лекций, на элективе читали лекции, на эле...","побороть « страх сцены »., каждые следующие пр..."
2,Психика и мозг,проводили много интересных тестов нервной сист...,"тест, лекция, сложный, онлайн, ответ, итог, мо...","электив, памяти, внимания, интересный, мозга, ...","интересных тестов нервной системы, обязательны...","после прохождения тестов по лекциям, электив о...","лабораторные ), выполняли различные задания, и..."
3,Психология общения,"лекции - нудные, скучные, там другой преподава...","сериал, посмотреть, практика, преподаватель, г...","деле изучаешь, преподаватель, лекции ходила к ...","других людях шикарный предмет, первую серию се...","предоставляются преподавателями всем советую, ...",которые предоставляются преподавателями всем с...
4,Фото- и видеоискусство,"прикольный был преподаватель, учил правилам на...","рассказывать, мы, семинар, фотоаппарат, лекция...","лекции, кстати почему написано видео искусство...","весенний период обучения, подборе цветовой гам...","кучу инфы на лекции но, учитывая предпочтения ...",кстати почему написано видео искусство остаётс...
...,...,...,...,...,...,...,...
127,Коммуникативная грамматика (английский язык),"изучаем времена, разговариваем, играем в игры....","играть, разговаривать, игра, домашка, изучать,...","домашки, игры, разговариваем, изучаем времена,","домашки, игры, времена,","нет домашки, игры нет, изучаем времена, игры н...","изучаем времена,"
128,Китайский для начинающих и продолжающих,"если вам нравятся восточные языки, то вы точно...","иероглиф, лаоши, восточный, достигать, запомин...","каких-то результатов, интересные, реально начи...","домашку(в основном, каких-то результатов, инте...","задания реально начинаешь запоминать фразы, де...","реально начинаешь запоминать фразы, яна лаоши ..."
129,Кинотекст: от искусства к арт-хаусу,"тематика прикольная, очень здорово, что ты смо...","неожиданный, тестирование, удобный, прикольный...","лекции, здорово, единственный минус, оказался ...",неожиданным - обязательное итоговое тестирован...,"время лекции потом уже, тестирование прокторин...","который оказался супер неожиданным, обязательн..."
130,Как читать книги,"электив, на котором придётся очень много работ...","изобразить, контекст, стихотворение, нарисоват...","котором, сложно описать, творческие задания, и...","анализа стихотворений, отдельной рамке, слово ...","рамке много анализа стихотворений, особенно пи...","каждое слово нужно изобразить, ставят баллы )...."


In [ ]:
predictions = {"Titles": [],
               "Feedbacks": [],
              "KeyBERT": []}
for el_index in el_indeces:
  predictions["Titles"].append(electives_concat_feedbacks["title"][el_index])
  predictions["Feedbacks"].append(electives_concat_feedbacks["concat_text"][el_index])
  for predictor_name, predictor_method in predictors.items():
    predictions[predictor_name].append(predictor_method(el_index))

predictions = pd.DataFrame(predictions)
predictions

,Titles,Feedbacks,KeyBERT
0,Литература фанфикшн: интерпретация классически...,"обсуждение канона, фанона, фанфиков, пейрингов...","взять другое дз, серьезно при прогулах занятий..."
1,Практическая риторика и теория аргументации,"преподаватель очень добрая девушка, любит свой...","печа куча тройная спираль, поставит нормальное..."
2,Психика и мозг,проводили много интересных тестов нервной сист...,"вшэ стоит начать, левому правому полушариям, к..."
3,Психология общения,"лекции - нудные, скучные, там другой преподава...","посмотрела все сезоны, не сильно энергозатратн..."
4,Фото- и видеоискусство,"прикольный был преподаватель, учил правилам на...","марата губайдуллина очень классные, не будет п..."
...,...,...,...
127,Коммуникативная грамматика (английский язык),"изучаем времена, разговариваем, играем в игры....","играем игры нет, игры нет домашки, изучаем вре..."
128,Китайский для начинающих и продолжающих,"если вам нравятся восточные языки, то вы точно...","каких то результатов нужно делать, вы точно не..."
129,Кинотекст: от искусства к арт-хаусу,"тематика прикольная, очень здорово, что ты смо...","прикольная очень здорово, прокторингом конце п..."
130,Как читать книги,"электив, на котором придётся очень много работ...","их проверяют за, тут сложно описать что именно..."


In [ ]:
predictions.to_csv("drive/MyDrive/ОБЩЕЕ/Отзывус/AI/key_tokens/data/feedbacks_tokens_predictions_KeyBERT_DeepPavlov.csv")